<a href="https://colab.research.google.com/github/efried130/Fusion_SNiP/blob/main/Master_Bands_EF_NietupskiFusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import os
import ee
# Trigger the authentication flow.
ee.Authenticate()
# Initialize the library.
ee.Initialize()

#!pip install geemap
#import geemap

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=Z5njFqhYPr7P8PhhYkcys6YwCsEF7YP83kR4tmD4Qxk&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWhZnd4qqs2TnvAQmNUpmS4MG-WPiHq0IqZZoX24eO7_FiFBRn3CtQo

Successfully saved authorization token.


In [26]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#!git clone https://github.com/tytupski/GEE-Image-Fusion.git

In [4]:
os.chdir('/content/gdrive/MyDrive/SNiP/RS_fusion/Nietupski_etal_code/GEE-Image-Fusion-Bands-EF')
! ls

Bands_Predict_L8_EF.py	GEE_ImageFusion  Master_Bands_EF_NietupskiFusion.ipynb


In [27]:
#Optional for if executing in colab cells
from GEE_ImageFusion.core_functions import *
from GEE_ImageFusion.get_paired_collections import *
from GEE_ImageFusion.prep_functions import *

In [ ]:
#Run from shell
#! python3 Bands_Predict_L8_EF.py

In [28]:
#Run in colab

#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Author: Ty Nietupski (ty.nietupski@oregonstate.edu)

This script shows an example of how the functions in the prep_functions,
get_paired_collections, and core_functions scripts can be applied to predict
all images from a specified timeframe at an individual scene location. To
use this code as a module, I've just placed the GEE_ImageFusion directory in
the site-packages directory of the relevant virtual environment so that
importing these functions into new script is easy.

General outline:
    1. Define global vars
    2. Organize dataset into nested lists
    3. Predict images in groups of 10 to avoid exceeding the memory limit.
        1. Register Landsat and MODIS
        2. Mask Landsat and MODIS and modify format for fusion (select similar
        pixels and convert images to 'neighborhood' images)
        3. Calculate the spatial and spectral distance to neighboring pixels
        and use to determine individual pixel weight.
        4. Perform regression over selected pixels to determine conversion
        coefficient.
        5. Use weights and conversion coefficient to predict images between
        image pairs.


The MIT License

Copyright © 2021 Ty Nietupski

Permission is hereby granted, free of charge, to any person obtaining a copy of
this software and associated documentation files (the "Software"), to deal in
the Software without restriction, including without limitation the rights to
use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies
of the Software, and to permit persons to whom the Software is furnished to do
so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.

"""

##############################################################################
# %% import
##############################################################################

import ee
from GEE_ImageFusion import *

ee.Initialize()

##############################################################################
# %% GLOBALS
##############################################################################

# region of interest
# location for prediction (outside scene overlap areas)
region = ee.Geometry.Point([-90.803, 38.907])

# # define training data temporal bounds broadly
startDate = '2021-02-01'
endDate = '2021-08-01'
#region = ee.Geometry.Point([-118.12627784505537, 44.66875751833964])

# define training data temporal bounds broadly
# startDate = '2017-03-01'
# endDate = '2018-01-01'
# common bands between sensors that will be used for fusion
#   would need to add functions for other indices (evi etc.).
#   ndvi is exported as 16 bit int so would have to make sure not to rescale
#   reflectance bands if that is what you are planning to predict (line 206)
#   some resturcturing of this code would be necessary if the goal was to
#   predict a multiband image but the core functions should work for any number
#   of bands
commonBandNames = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])

# image collections to use in fusion
# NOTE: if using older Landsat and not using NDVI one would have to modify the
# get_paired_collections script because this script harmonizes NDVI from
# L5 & L7 to L8 based on Roy et al. 2016 (see etmToOli and getPaired functions)
landsatCollection = 'LANDSAT/LC08/C01/T1_SR' #deprecated
modisCollection = 'MODIS/006/MCD43A4'       

# landsat band names including qc band for masking
bandNamesLandsat = ee.List(['blue', 'green', 'red',
                            'nir', 'swir1', 'swir2', 'pixel_qa'])
landsatBands = ee.List([1, 2, 3, 4, 5, 6, 10])

# modis band names
bandNamesModis = ee.List(['blue', 'green', 'red', 'nir', 'swir1', 'swir2'])
modisBands = ee.List([2, 3, 0, 1, 5, 6])

# radius of moving window
# Note: Generally, larger windows are better but as the window size increases,
# so does the memory requirement and we quickly will surpass the memory
# capacity of a single node (in testing 13 was max size for single band, and
# 10 was max size for up to 6 bands)
kernelRadius = ee.Number(10)
kernel = ee.Kernel.square(kernelRadius)
numPixels = kernelRadius.add(kernelRadius.add(1)).pow(2)

# number of land cover classes in scene
coverClasses = 7

# to export the images to an asset we need the path to the assets folder
path = 'users/efriedmann/'
scene_name = 'NDVI_PNW_onlyInt_'

##############################################################################
# %% get filtered collections
##############################################################################

# sorted, filtered, paired image retrieval
paired = getPaired(startDate, endDate,
                   landsatCollection, landsatBands, bandNamesLandsat,
                   modisCollection, modisBands, bandNamesModis,
                   commonBandNames, region)
print(paired[0].getInfo())
print(paired[0].propertyNames().getInfo())
print(paired[1].getInfo())
print(paired[2].getInfo())
subs = makeSubcollections(paired)
subs_meta = subs.getInfo()
print(subs_meta)

##############################################################################
# %% Predict and Export Images
##############################################################################

# loop through each list of paired images
num_lists = subs.length().getInfo()

for i in range(0, num_lists):
    # determine the number of modis images between pairs
    num_imgs = ee.List(ee.List(subs.get(i)).get(2)).length()
    print(num_imgs.getInfo())
    # determine the remainder of images, if not groups of 10
    remaining = num_imgs.mod(5)
    print(remaining.getInfo())
    # create sequence of starting indices for modis images
    index_seq = ee.List.sequence(0, num_imgs.subtract(remaining), 5)
    print(index_seq.getInfo())
    # images to be grouped and predicted
    subList = ee.List(ee.List(subs.get(i)).get(2))
    print(subList.getInfo())
    # loop through indices predicting in batches of 10
    for x in range(0, index_seq.length().getInfo()):
        # starting index
        start = ee.Number(index_seq.get(x))

        # ending index
        end = ee.Algorithms.If(start.add(5).gt(num_imgs),
                               num_imgs,
                               start.add(5))

        # group of images to predict
        pred_group = subList.slice(start, end)
        #print(pred_group.getInfo())
        landsat_t01 = ee.List(ee.List(subs.get(i)).get(0))
        #print(landsat_t01.getInfo())
        modis_t01 = ee.List(ee.List(subs.get(i)).get(1))
        #print(landsat_t01.getInfo())
        modis_tp = pred_group
        #print(modis_tp.getInfo())
        
        # get the start and end day values and year for the group to use
        # to label the file when exported to asset
        startDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                   .first()
                                   .get('DOY'))
        endDay = ee.Number.parse(ee.ImageCollection(pred_group)
                                 .sort('system:time_start', False)
                                 .first()
                                 .get('DOY'))
        year = ee.Date(ee.ImageCollection(pred_group)
                       .sort('system:time_start', False)
                       .first()
                       .get('system:time_start')).format('Y')

        # start and end day of year
        doys = landsat_t01 \
            .map(lambda img: ee.String(ee.Image(img).get('DOY')).cat('_'))

        # register images
        landsat_t01, modis_t01, modis_tp = registerImages(landsat_t01,
                                                          modis_t01,
                                                          modis_tp)
        print(modis_tp.getInfo())
        # prep landsat imagery (mask and format)
        maskedLandsat, pixPositions, pixBN = prepLandsat(landsat_t01,
                                                         kernel,
                                                         numPixels,
                                                         commonBandNames,
                                                         doys,
                                                         coverClasses)
        
        # prep modis imagery (mask and format)
        modSorted_t01, modSorted_tp = prepMODIS(modis_t01, modis_tp, kernel,
                                                numPixels, commonBandNames,
                                                pixBN)
        
        # calculate spectral distance
        specDist = calcSpecDist(maskedLandsat, modSorted_t01,
                                numPixels, pixPositions)

        # calculate spatial distance
        spatDist = calcSpatDist(pixPositions)
        
        # calculate weights from the spatial and spectral distances
        weights = calcWeight(spatDist, specDist)
        
        # calculate the conversion coefficients
        coeffs = calcConversionCoeff(maskedLandsat, modSorted_t01,
                                     doys, numPixels, commonBandNames)
        
        # predict all modis images in modis tp collection
        prediction = modSorted_tp \
            .map(lambda image:
                 predictLandsat(landsat_t01, modSorted_t01,
                                doys, ee.List(image),
                                weights, coeffs,
                                commonBandNames, numPixels))

        # create a list of new band names to apply to the multiband image
        # NOTE: cant export with names starting with 0
        predsColl = ee.ImageCollection(prediction)
        print(predsColl.getInfo())
        preds = ee.ImageCollection(prediction).toBands()
        dates = modis_tp.map(lambda img:
                             ee.Image(img).get('system:time_start'))
        predNames = ee.List.sequence(0, prediction.length().subtract(1)) \
            .map(lambda i:
                 commonBandNames\
                     .map(lambda name:
                          ee.String(name)
                          .cat(ee.String(ee.Number(dates.get(i)).format()))))\
            .flatten()
        
        # export all predictions as a single multiband image
        # each band name corresponds to the timestamp for the image
        task = ee.batch.Export.image.toAsset(
            image=preds.rename(predNames).toInt16(),#multiply(10000).toInt16(),
            description=ee.String(scene_name)
                        .cat(year)
                        .cat('_')
                        .cat(startDay.format())
                        .cat('_').cat(endDay.format()).getInfo(),
            assetId=ee.String(path)
                    .cat(ee.String(scene_name))
                    .cat(year)
                    .cat('_')
                    .cat(startDay.format())
                    .cat('_')
                    .cat(endDay.format()).getInfo(),
            region=ee.Image(prediction.get(0)).geometry(),
            scale=30)

        #task.start()

{'type': 'ImageCollection', 'bands': [], 'id': 'LANDSAT/LC08/C01/T1_SR', 'version': 1646556232374811, 'properties': {'system:visualization_0_min': '0.0', 'type_name': 'ImageCollection', 'keywords': ['cfmask', 'cloud', 'fmask', 'global', 'l8sr', 'landsat', 'lc08', 'reflectance', 'sr', 'usgs'], 'visualization_1_bands': 'B5,B4,B3', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_1_max': '30000.0', 'description': '<p>This dataset is the atmospherically corrected\nsurface reflectance from  the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and two thermal infrared (TIR) bands processed to orthorectified\nbrightness temperature</p><p>These data have been atmospherically corrected using\n<a href="https://prd-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/atoms/files/LSDS-1368_L8_C1-LandSurfaceReflectanceCode-LAS

In [31]:
#Extract the multiband image
preds16=preds.rename(predNames).toInt16()
print(type(preds16))
print(preds16.bandNames().getInfo())
preds16_1 = preds16.slice(0, 6)
type(preds16_1)

<class 'ee.image.Image'>
['blue1623369600000', 'green1623369600000', 'red1623369600000', 'nir1623369600000', 'swir11623369600000', 'swir21623369600000', 'blue1623456000000', 'green1623456000000', 'red1623456000000', 'nir1623456000000', 'swir11623456000000', 'swir21623456000000']


ee.image.Image

In [42]:
samples = preds16_1.sample(
  region = preds16.geometry(),

  #Default (false) is no geometries in the output.
  #When set to true, each feature has a Point geometry at the center of the
  #image pixel.
  geometries = True,

  #The scale is not specified, so the resolution of the image will be used,
  #and there is a feature for every pixel. If we give a scale parameter, the
  #image will be resampled and there will be more or fewer features.

  scale = 30,
)
print(samples.getInfo())

#Break point coordinates up into properties (table columns) explicitly.
def valueCoords(feature):
  coordinates = feature.geometry().transform('epsg:4326').coordinates()
  return feature.set('lon', coordinates.get(0), 'lat', coordinates.get(1))

collection_with_latlon = samples.map(valueCoords)
print(collection_with_latlon.getInfo())

EEException: ignored

In [ ]:
# Extract an image collection
#print(type(predsColl))
print(predsColl.getInfo())
predsColl = ee.ImageCollection(predsColl.map(lambda image: ee.Image(image).multiply(10000).toInt16()))

image1 = predsColl.first()
print(image1.getInfo())

{'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'blue', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7861, 7991], 'origin': [10, 0], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 281985, 0, -30, 5059515]}, {'id': 'green', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7861, 7991], 'origin': [10, 0], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 281985, 0, -30, 5059515]}, {'id': 'red', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7861, 7991], 'origin': [10, 0], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 281985, 0, -30, 5059515]}, {'id': 'nir', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7861, 7991], 'origin': [10, 0], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 281985, 0, -30, 5059515]}, {'id': 'swir1', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7861, 7991], 'origin': [10, 0], 'crs': 'EPSG:32611', 

In [ ]:
#Landsat image at 5/12/21
landsatUpdated = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR').filterBounds(region).filterDate('2021-05-12', '2021-09-20');
land512 = landsatUpdated.first().slice(0,6)
print(land512.getInfo())

{'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7871, 7991], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 282285, 0, -30, 5059515]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7871, 7991], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 282285, 0, -30, 5059515]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7871, 7991], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 282285, 0, -30, 5059515]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7871, 7991], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 282285, 0, -30, 5059515]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -32768, 'max': 32767}, 'dimensions': [7871, 7991], 'crs': 'EPSG:32611', 'crs_transform': [30, 0, 2822

In [ ]:
#Get the AD of two images

diff_AD = land512.subtract(image1).abs();
print('AD Land-Niet', diff_AD.getInfo());

AD Land-Niet {'type': 'Image', 'bands': [{'id': 'B1', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7561, 7741], 'origin': [30, 0], 'crs': 'EPSG:32615', 'crs_transform': [30, 0, 578085, 0, -30, 4424115]}, {'id': 'B2', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7561, 7741], 'origin': [30, 0], 'crs': 'EPSG:32615', 'crs_transform': [30, 0, 578085, 0, -30, 4424115]}, {'id': 'B3', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7561, 7741], 'origin': [30, 0], 'crs': 'EPSG:32615', 'crs_transform': [30, 0, 578085, 0, -30, 4424115]}, {'id': 'B4', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7561, 7741], 'origin': [30, 0], 'crs': 'EPSG:32615', 'crs_transform': [30, 0, 578085, 0, -30, 4424115]}, {'id': 'B5', 'data_type': {'type': 'PixelType', 'precision': 'double'}, 'dimensions': [7561, 7741], 'origin': [30, 0], 'crs': 'EPSG:32615', 'crs_transform': [30, 0, 578085, 0, -30, 4424115]},

In [ ]:
#Method 1: Calculate MSE by adding the orginal landsat to a feature collection

#Define region, image collection, and original image
#region is above
predsColl_rmse = predsColl.map(lambda image: ee.Image(image.select(['blue.*', 'red']).addBands(land512.select(['B1']))))
predsColl_rmse_first = predsColl_rmse.first()
#Function for the Mean Squared Difference
def sqDiff(img):
  SubSq = img.select(['B1']).subtract(img.select(['blue.*'])).pow(2)
  return ee.Feature(region, SubSq.reduceRegion(ee.Reducer.mean(), region))

#Produce a feature collection of RMSE
mseCollection = predsColl_rmse.map(sqDiff)
print(mseCollection.getInfo())

{'type': 'FeatureCollection', 'columns': {}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-118.12627784505537, 44.66875751833964]}, 'id': '0', 'properties': {'B1': 1059111936}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-118.12627784505537, 44.66875751833964]}, 'id': '1', 'properties': {'B1': 1059111936}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-118.12627784505537, 44.66875751833964]}, 'id': '2', 'properties': {'B1': 1059111936}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-118.12627784505537, 44.66875751833964]}, 'id': '3', 'properties': {'B1': 1059111936}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-118.12627784505537, 44.66875751833964]}, 'id': '4', 'properties': {'B1': 1059111936}}]}


In [ ]:
#Method 2: too big and user memory limit exceeded
land512_blue = land512.select('B1')

preds16_blue = preds16.select('blue1622937600000')

differenceSquared = land512_blue.subtract(preds16_blue).pow(2)

RMSE = differenceSquared.reduceRegion(reducer = ee.Reducer.mean(), scale = 100, bestEffort = True)
print(RMSE.getInfo())

In [ ]:
#Method 3: Using arrays
ArrLand512_blue = ee.Array(land512_blue.toArray())
ArrPreds16_blue = ee.Array(preds16_blue.toArray())

#Calculate the RMSE
RMSE = (ArrLand512_blue.subtract(ArrPreds16_blue)).pow(2)       # array calculations
print(type(RMSE))
RMSE = RMSE.reduce('sum', [0]).get([0])  # reduce the array to the sum -> output is a ee.Number()
print(RMSE)
RMSE = RMSE.sqrt()                      # Get the RMSE
print(type(RMSE))
RMSE = RMSE.divide(ArrLand512_blue.length().get([0]))    # divide by the amount of observations
print(type(RMSE))

print(RMSE)


NameError: ignored

In [ ]:
Map = geemap.Map(center=[38.907, -90.803], zoom = 6)

#image = ee.Image('users/efriedmann/NDVI_P24R33_allBands_2021_137_146')

#vizParams = {
#     'bands': ["blue1621209600000", "green1621209600000", "red1621209600000"],
#     'min': 0, 'max': 32767
# }

Map.addLayer(diff_AD, {'min': 0, 'max': 32767}, 'PredsColl')
Map.addLayerControl()
Map

Map(center=[38.907, -90.803], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

In [ ]:
# Import the Folium library.
import folium
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name, mapTypeId='SATELLITE'):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  #'bands': ['B3'],
  'min': 0,
  'max': 3000}#,
  #'palette': ['blue', 'red', 'green']}

# Create a folium map object.
my_map = folium.Map(location=[38.907, -90.803], zoom_start=8)

# Add the elevation model to the map object.
my_map.add_ee_layer(image1, vis_params, 'red')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)